In [1]:
#Import all libraries
from collections import defaultdict
from collections import  defaultdict
import math
import random

In [2]:
#Get tokens (You may need to launch the functions in tokenizer_nltk first as they will create the files we use now)
with open('../tokenizer_nltk/data/training_data/whitespace_tokenizer_train.data', 'r') as f:
    train = f.readlines()
    train = [word.strip() for word in train]

with open('../tokenizer_nltk/data/testing_data/whitespace_tokenizer_test.data', 'r') as f:
    test = f.readlines()
    test = [word.strip() for word in test]


In [3]:
def Create_And_Count_ngrams(tokens, n):
    # Creates n-grams and counts their occurrences
    ngram_counts = defaultdict(int)
    
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i+n])
        ngram_counts[ngram] += 1
    
    return ngram_counts
    

def add_laplace(ngram, k):
    # Applies Laplace smoothing 
    for key in ngram:
        ngram[key] += k

def Create_ngram(train, test, n, k=0.00001):
# Creates the needed n-gram
    
    token_set = set(train)  # To not have duplicate tokens
    len_token_set = len(token_set)
    
    # Step 1: Count n-grams and (n-1)-grams 
    tmp_ng_with_count = Create_And_Count_ngrams(train, n)
    prefix_count = Create_And_Count_ngrams(train, n - 1)
    final_ng = defaultdict(float)

    # Step 2: Apply Laplace smoothing
    add_laplace(tmp_ng_with_count, k)
    add_laplace(prefix_count, k)
   
    # Step 3: Calculate n-gram probabilities
    for ng in tmp_ng_with_count:
        final_ng[ng] = (tmp_ng_with_count[ng] + k) / (prefix_count[ng[:-1]] + k * len_token_set)

    # Step 4: Fixing unseen data that exists in the test set
    for i in range(len(test) - n + 1):
        ng = tuple(test[i:i + n])
        if ng not in tmp_ng_with_count:
            final_ng[ng] = k / (prefix_count[ng[:-1]] + k * len_token_set)
    return final_ng


def calculate_perplexity(test, ngram, n):
    # Calculates perplexity
    log_probability_sum = 0
    ngram_count = 0
    
    for i in range(len(test)-n+1):
        log_probability_sum += math.log2(ngram[tuple(test[i:i+n])])
        ngram_count += 1
    
    average_log_probability = -log_probability_sum / ngram_count
    perplexity = math.pow(2, average_log_probability)
    
    return perplexity


def greedy_sampling(context, ngram_probabilities, n, max_length=50):
    # Take the most probable option only
    sentence = []
    sentence.extend(tuple(context[-(n):]))

    # Check is context toot small
    if len(context) < (n-1):
        print("context too small")
        return sentence

    context = tuple(context[-(n-1):])  
    
    for _ in range(max_length):
        probs = {}
        
        for token in ngram_probabilities:
            if token[:n-1] == context:
                probs[token[-1]] = ngram_probabilities[token]

        if not probs:
            print("No token possible in context")
            return sentence

        # For greedy algorithm
        best_token = max(probs, key=probs.get)  

        if best_token not in probs:
            print("No best token possible")
            return sentence

        sentence.append(best_token)
        context = context[1:] + (best_token,) 

    return sentence

import random

def top_k(context, ngram_probabilities, n, max_length=50, k=1):
    # Top k version. if k = 1, same result as greedy
    sentence = []
    sentence.extend(tuple(context[-(n):]))

    # Check if context too small
    if len(context) < (n-1):
        print("context too small")
        return sentence

    context = tuple(context[-(n-1):])  
    
    for _ in range(max_length):
        probs = {}
        
        for token in ngram_probabilities:
            if token[:n-1] == context:
                probs[token[-1]] = ngram_probabilities[token]

        if not probs:
            print("No token possible in context")
            return sentence

        # Selecting top k tokens probabilistically
        top_k_tokens = sorted(probs, key=probs.get, reverse=True)[:k]

        token_weights = [probs[token] for token in top_k_tokens]
        best_token = random.choices(top_k_tokens, weights=token_weights, k=1)[0]


        if best_token not in probs:
            print("No best token possible")
            return sentence

        sentence.append(best_token)
        context = context[1:] + (best_token,) 

    return sentence



In [8]:
N_Gram = Create_ngram(train,test,2)
N_Gram

defaultdict(float,
            {('I', 'currently'): 0.00014404813575830184,
             ('currently', 'have'): 0.053714998264261446,
             ('have', 'been'): 0.08319134248590251,
             ('been', 'using'): 0.07784564557526263,
             ('using', 'the'): 0.12077639597408907,
             ('the', '"Hemp'): 3.8277529937516656e-06,
             ('"Hemp', 'Pro'): 0.32234689634498165,
             ('Pro', '50"'): 0.012970039419052427,
             ('50"', 'only'): 0.24376997350273386,
             ('only', 'because'): 0.008957593889017998,
             ('because', 'the'): 0.08131220869633805,
             ('the', 'local'): 0.0028248252894379996,
             ('local', 'Whole'): 0.008206138436102371,
             ('Whole', 'Foods'): 0.3910334075867565,
             ('Foods', 'did'): 0.004493393889598102,
             ('did', 'not'): 0.4045600936361317,
             ('not', 'carry'): 0.0014943055227355068,
             ('carry', 'the'): 0.0804184034817246,
             ('the', 

In [9]:
calculate_perplexity(train,N_Gram,2)

98.12042657763496

In [10]:
calculate_perplexity(test,N_Gram,2)

609.7416033316772

In [7]:
context = ['I', 'currently', 'have','been']
print(top_k(context, N_Gram,5, 30,3))


['I', 'currently', 'have', 'been', 'using', 'the', 'Sumatran', 'for', 'a', 'while', 'but', 'then', 'got', 'burned', 'out', 'on', 'them.', 'They', 'are', 'a', 'bit', 'pricey', 'when', 'you', 'factor', 'in', 'shipping.<br', '/><br', '/>BTW,', 'the', 'package', 'of', 'ten', '1.5']
